In [ ]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

from tensorflow.keras.layers import Conv1D, SpatialDropout1D, BatchNormalization, GlobalAveragePooling1D, Dense
from tensorflow.keras.layers import Reshape,Flatten,BatchNormalization,MaxPooling1D,AveragePooling2D,Reshape,Attention, ReLU, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold
import keras.backend as K
from tensorflow.keras.optimizers import Adam
from tqdm.auto import tqdm
import os
from Config import Config
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Mean, CategoricalAccuracy
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from datetime import datetime
import random
import matplotlib.pyplot as plt
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.layers import Conv1D, SpatialDropout1D, BatchNormalization
from tensorflow.keras.activations import sigmoid

import tensorflow as tf
tf.config.run_functions_eagerly(True)

import warnings
warnings.filterwarnings('always')


In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
seed_everything()

In [ ]:
DATA_ROOT = 'TIMNET-dataset'
DATA_PATH = 'EMODB'
CLASS_LABELS = Config.EMODB_LABELS
k = 10

model_name = 'TIMNet'
feature_name = 'mfcc'

EPOCHS = 300
BATCH = 64

In [ ]:
class TAB(tf.keras.Model):
  def __init__(self, i):
    super(TAB, self).__init__()
    
    dilation_rate = 2**(i-1)
    
    self.conv1 = Conv1D(39, kernel_size=1, strides=1, padding='same', dilation_rate=(dilation_rate))
    self.bn1 = BatchNormalization()
    self.spatial_drop1 = SpatialDropout1D(0.1)
    self.conv2 = Conv1D(39, kernel_size=1, strides=1, padding='same', dilation_rate=(dilation_rate))
    self.bn2 = BatchNormalization()
    self.spatial_drop2 = SpatialDropout1D(0.1)

  def call(self, inputs):
    x = self.conv1(inputs)
    x = self.bn1(x)
    x = self.spatial_drop1(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.spatial_drop2(x)
    x = tf.keras.activations.sigmoid(x)
    
    output = inputs * x
    
    return x

In [ ]:
class TIMNet(tf.keras.Model):

  def __init__(self, num_classes):
    super(TIMNet, self).__init__()
    
    ## forward TAB
    self.f_tab1 = TAB(1)
    self.f_tab2 = TAB(2)
    self.f_tab3 = TAB(3)
    self.f_tab4 = TAB(4)
    self.f_tab5 = TAB(5)
    
    ## backward TAB
    self.b_tab1 = TAB(1)
    self.b_tab2 = TAB(2)
    self.b_tab3 = TAB(3)
    self.b_tab4 = TAB(4)
    self.b_tab5 = TAB(5)
    
    self.gap = GlobalAveragePooling1D()
    self.fc = Dense(num_classes, activation="softmax")

  def call(self, inputs):
    forward = inputs
    backward = tf.reverse(inputs, axis=[1])
    
    f1 = self.f_tab1(forward)
    b1 = self.b_tab1(backward)
    g1 = self.gap(f1) + self.gap(b1)
    
    f2 = self.f_tab2(f1)
    b2 = self.b_tab2(b1)
    g2 = self.gap(f2) + self.gap(b2)
    
    f3 = self.f_tab2(f2)
    b3 = self.b_tab3(b2)
    g3 = self.gap(f3) + self.gap(b3)
    
    f4 = self.f_tab2(f3)
    b4 = self.b_tab3(b3)
    g4 = self.gap(f4) + self.gap(b4)
    
    f5 = self.f_tab2(f4)
    b5 = self.b_tab3(b4)
    g5 = self.gap(f5) + self.gap(b5)
    
    dynamic_fusion = g1+g2+g3+g4+g5
    output = self.fc(dynamic_fusion)
    
    return output

In [ ]:
model = TIMNet(len(CLASS_LABELS))
model.build((64, 182, 39))
model.summary()

In [ ]:
@tf.function
def train_step(model, optimizer, x, labels):
    with tf.GradientTape() as tape:
        # 미분 계산
        predictions = model(x, training=True)
        loss = CategoricalCrossentropy()(labels, predictions)
        
    grad = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grad, model.trainable_variables))     # 신경망 파라미터 업데이트
    
    acc = tf.keras.metrics.CategoricalAccuracy()
    acc.update_state(labels, predictions)
    accuracy = acc.result().numpy()
    
    return loss, accuracy

def test_step(model, x, labels):
    predictions = model(x)
    loss = CategoricalCrossentropy()(labels, predictions)
    
    acc = tf.keras.metrics.CategoricalAccuracy()
    acc.update_state(labels, predictions)
    accuracy = acc.result().numpy()
    
    return loss, accuracy*100, predictions

In [ ]:
# Read data
with open(f'{DATA_ROOT}/{DATA_PATH}.npy', 'rb') as f:
    x = np.load(f)
    y = np.load(f)

y = to_categorical(y,num_classes=len(CLASS_LABELS))

In [ ]:
# Smooth label operation
def smooth_labels(labels, factor=0.1):
    """
        smooth the labels
        returned the smoothed labels
    """
    labels *= (1 - factor)
    labels += (factor / labels.shape[1])
    return labels

In [ ]:
from discord_notice import start, end
start()

In [ ]:
import tensorflow_addons as tfa
from keras.models import load_model

emotions_groundtruth_list = np.array([])
predicted_emotions_list = np.array([])

avg_acc = 0.0

kfold = KFold(n_splits=k, shuffle=True, random_state=98)
for i, (train, test) in tqdm(enumerate(kfold.split(x, y)), desc=f'Training {k}-Fold.....'):
    now_time = datetime.now().strftime("%m-%d-%H%M%S")
    
    save_path = f'Models/{DATA_PATH}'
    os.makedirs(save_path, exist_ok=True)
    h5_path = f'{save_path}/{model_name}_{feature_name}_{i}-fold_{now_time}.h5'
    
    x_train, y_train = x[train], y[train]
    y_train = smooth_labels(y[train], 0.1)
    
    x_test, y_test = x[test], y[test]
    
    shape = x_train.shape[1:]
    
    optimizer = Adam(learning_rate=0.001)
    
    model = TIMNet(len(CLASS_LABELS))
    
    best_test_loss = 0x3f3f3f
    best_test_acc = -1
    best_test_f1 = -1
    
    epoch_losses = []
    valid_losses = []
    
    batch_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(46).batch(BATCH)
    batch_test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH)
    for epoch in tqdm(range(EPOCHS), desc=f'Fold-{i+1}'):

        train_loss, train_acc = [], []
        for features, labels in batch_train:
            loss, acc = train_step(model, optimizer, features, labels)
            train_loss.append(loss)
            train_acc.append(acc)
            
        test_loss, test_acc, f1s = [], [], []
        for features, labels in batch_test:
            loss, acc, pred = test_step(model, features, labels)
            test_loss.append(loss)
            test_acc.append(acc)
            
            f1_metric = tfa.metrics.F1Score(num_classes=len(CLASS_LABELS), average='weighted')
            f1_metric.update_state(labels, pred)
            f1 = f1_metric.result().numpy()
            f1s.append(f1)
            
        
        epoch_loss = sum(train_loss)/len(train_loss)
        epoch_acc = sum(train_acc)/len(train_acc)
        val_loss = sum(test_loss)/len(test_loss)
        val_acc = sum(test_acc)/len(test_acc)
        f1_score = sum(f1s)/len(f1s)
        
        epoch_losses.append(epoch_loss)
        valid_losses.append(val_loss)
        
        cur_lr = K.eval(optimizer.lr)
        print(f'{epoch+1}/{EPOCHS} lr={cur_lr:.5f} - loss:{epoch_loss:.3f}, acc:{epoch_acc:.3f}, val_loss:{val_loss:.3f}, val_acc:{val_acc:.3f}')
        print(f'Best loss:{best_test_loss:.3f}, Best accuracy:{best_test_acc:.3f}, Best F1-score:{best_test_f1:.3f}')
        
        if best_test_acc < val_acc:
            best_test_acc = val_acc
            best_test_loss = val_loss
            best_test_f1 = f1_score
            model.save_weights(h5_path)
            
            
    model = TIMNet(len(CLASS_LABELS))
    model.build(input_shape=x_train.shape)
    model.load_weights(h5_path)
    best_pred = model(x_test, training=False)
    emotions_groundtruth_list = np.append(emotions_groundtruth_list, np.argmax(y_test, axis=1))
    predicted_emotions_list = np.append(predicted_emotions_list, np.argmax(best_pred, axis=1))
    
    
    print(f'[*] Done - acc:{best_test_acc:.3f}')
    
    plt.title('Loss Curve')
    plt.ylabel('Loss', fontsize=16)
    plt.xlabel('Epoch', fontsize=16)
    plt.plot(epoch_losses[:],'b')
    plt.plot(valid_losses[:],'r')
    plt.legend(['Training loss','Validation loss'])
    plt.show()
    
    avg_acc += best_test_acc
    

In [ ]:
end()

In [ ]:
avg_acc/k

In [ ]:
import warnings
warnings.filterwarnings('always')

Report = classification_report(emotions_groundtruth_list, predicted_emotions_list)

os.makedirs(f'Results/{DATA_PATH}', exist_ok=True)
report_path = f'Results/{DATA_PATH}/{model_name}_{feature_name}_{k}-fold_nomalize.txt'

with open(report_path, "w") as f:
    f.write(Report)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

emotion_names = CLASS_LABELS


# build confusion matrix and normalized confusion matrix
conf_matrix = confusion_matrix(emotions_groundtruth_list, predicted_emotions_list)
conf_matrix_norm = confusion_matrix(emotions_groundtruth_list, predicted_emotions_list,normalize='true')

# make a confusion matrix with labels using a DataFrame
confmatrix_df = pd.DataFrame(conf_matrix, index=emotion_names, columns=emotion_names)
confmatrix_df_norm = pd.DataFrame(conf_matrix_norm, index=emotion_names, columns=emotion_names)

# plot confusion matrices
plt.figure(figsize=(16,6))
sn.set(font_scale=1.8) # emotion label and title size
plt.subplot(1,2,1)
plt.title('Confusion Matrix')
sn.heatmap(confmatrix_df, annot=True, annot_kws={"size": 13}, fmt='g') #annot_kws is value font
plt.subplot(1,2,2)
plt.title('Normalized Confusion Matrix')
sn.heatmap(confmatrix_df_norm, annot=True, annot_kws={"size": 13}) #annot_kws is value font
plt.savefig(f'Results/{DATA_PATH}/{model_name}_{feature_name}_{k}-fold_confmatrix.pdf')
plt.show()